## 네이버 금융 개별종목 수집

1. 수집 하고자 하는 페이지의 URL을 알아본다.
1. 파이썬의 작은 브라우저 requests 를 통해 URL에 접근한다.
1. response.status_code 가 200 OK 라면 정상 응답입니다.
1. request의 response 값에서 response.text 만 받아옵니다.
1. html 텍스트를 받아왔다면  BeautifulSoup 로 html 을 해석합니다.
1. soup.select 를 통해 원하는 태그에 접근합니다.
1. 목록을 먼저 받아옵니다.
1. 목록에서 행을 하나씩 가져옵니다.
1. 행을 모아서 데이터프레임으로 만듭니다.
=> 판다스라면 read_html로 table로 된 태그에 대해 위 과정을 코드 한 줄로 해줍니다.



### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 주요 종목
    * 삼성전자 : https://finance.naver.com/item/main.naver?code=005930
    * 현대차 : https://finance.naver.com/item/main.naver?code=005380
    * SK하이닉스 : https://finance.naver.com/item/main.naver?code=000660

## 라이브러리 로드

In [30]:
# 라이브러리 로드
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup as bs

## 수집할 URL 정하기

In [12]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
#item_code = "323410"
#item_name = "카카오뱅크"

item_code = "005930"
item_name = "삼성전자"

# 종목 URL 만들기
# url
url =  f'https://finance.naver.com/item/sise_day.naver?code={item_code}&page=1'
print(url)

https://finance.naver.com/item/sise_day.naver?code=005930&page=1


## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [28]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.47'}

response = requests.get(url, headers=headers)
print(headers['user-agent'])
response.status_code

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.47


200

In [106]:
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

In [33]:
html = bs(response.text, 'lxml')
html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="

In [42]:
html.title

<title>네이버 금융</title>

In [43]:
html.title.string

'네이버 금융'

In [36]:
html.title.name

'title'

In [40]:
html.title.parent

<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/world.css" rel="stylesheet" type="text/css"/>
</head>

In [41]:
html.title.parent.name

'head'

In [45]:
len(html.find('table'))

33

In [47]:
len(html.find_all('table'))

2

In [52]:
tables = html.select('table')
len(tables)

2

## pandas 코드 한 줄로 데이터 수집하기

In [54]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
# table
table = pd.read_html(str(tables),encoding='cp949')
print(len(table))
table[0]

2


,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022.05.17,67800.0,1500.0,66600.0,67900.0,66600.0,11567705.0
2,2022.05.16,66300.0,200.0,67100.0,67400.0,66100.0,11937555.0
3,2022.05.13,66500.0,1600.0,65300.0,66700.0,65200.0,14551536.0
4,2022.05.12,64900.0,800.0,65200.0,65500.0,64900.0,16414188.0
5,2022.05.11,65700.0,0.0,65500.0,66300.0,65200.0,12330920.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2022.05.10,65700.0,400.0,65900.0,66300.0,65300.0,17235605.0


In [59]:
table = pd.read_html(requests.get(url, headers = headers).text, encoding='cp949')
print(len(table))

2


In [60]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022.05.17,67800.0,1500.0,66600.0,67900.0,66600.0,12889934.0
2,2022.05.16,66300.0,200.0,67100.0,67400.0,66100.0,11937555.0
3,2022.05.13,66500.0,1600.0,65300.0,66700.0,65200.0,14551536.0
4,2022.05.12,64900.0,800.0,65200.0,65500.0,64900.0,16414188.0
5,2022.05.11,65700.0,0.0,65500.0,66300.0,65200.0,12330920.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2022.05.10,65700.0,400.0,65900.0,66300.0,65300.0,17235605.0


In [64]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
# temp
temp = table[0].dropna(axis=0).reset_index(drop=True)
temp

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.05.17,67800.0,1500.0,66600.0,67900.0,66600.0,12889934.0
1,2022.05.16,66300.0,200.0,67100.0,67400.0,66100.0,11937555.0
2,2022.05.13,66500.0,1600.0,65300.0,66700.0,65200.0,14551536.0
3,2022.05.12,64900.0,800.0,65200.0,65500.0,64900.0,16414188.0
4,2022.05.11,65700.0,0.0,65500.0,66300.0,65200.0,12330920.0
5,2022.05.10,65700.0,400.0,65900.0,66300.0,65300.0,17235605.0
6,2022.05.09,66100.0,400.0,66300.0,66900.0,66100.0,11858736.0
7,2022.05.06,66500.0,1400.0,67000.0,67100.0,66500.0,14356156.0
8,2022.05.04,67900.0,400.0,68000.0,68400.0,67500.0,11505248.0
9,2022.05.03,67500.0,200.0,67400.0,68400.0,67300.0,14168875.0


## 페이지별 데이터 수집 함수 만들기

In [85]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# 일자별 시세를 페이지별로 수집

def get_day_list(item_code, page_no):
    # header definition
    headers = {'user-agent': 'Mozilla/5.0'}
    
    # url crawling
    url =  f'https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}'
    table = pd.read_html(requests.get(url,headers = headers).text, encoding='cp949')
    
    # df create & preprocessing
    df = table[0]
    df.dropna(inplace=True)
    df.reset_index(inplace=True, drop=True)
    
    return df

In [86]:
# 함수가 잘 만들어졌는지 확인하기
item_code = "005930"
item_name = "삼성전자"

get_day_list(item_code,3)

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.04.18,66700.0,100.0,66500.0,67100.0,66100.0,10119203.0
1,2022.04.15,66600.0,900.0,67200.0,67300.0,66500.0,13176415.0
2,2022.04.14,67500.0,1200.0,68700.0,68700.0,67500.0,16409494.0
3,2022.04.13,68700.0,1700.0,67300.0,69000.0,67200.0,17378619.0
4,2022.04.12,67000.0,900.0,67600.0,67700.0,67000.0,13924389.0
5,2022.04.11,67900.0,100.0,67800.0,68100.0,67400.0,12263735.0
6,2022.04.08,67800.0,200.0,68100.0,68300.0,67700.0,15453191.0
7,2022.04.07,68000.0,500.0,68500.0,68500.0,68000.0,20683327.0
8,2022.04.06,68500.0,700.0,68600.0,68800.0,68500.0,15517308.0
9,2022.04.05,69200.0,100.0,69400.0,69600.0,69100.0,8467248.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [112]:
import time

def get_days_list(item_code):
    # header definition
    headers = {'user-agent': 'Mozilla/5.0'}
    
    # url crawling
    table_list = []
    page=1
    prev=''
    while True:
        url =  f'https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page}'
        r = requests.get(url, headers=headers)
        if r.text == prev :
            break
        table = pd.read_html(r.text, encoding='cp949')[0]
        table_list.append(table)
        print(page, end=' ')
        time.sleep(0.1)
        prev = r.text
        page+=1
            
    # df create & preprocessing
    df = pd.concat(table_list)
    df.dropna(axis=0, inplace=True)
    df.reset_index(inplace=True, drop=True)
    
    return df

In [113]:
# 함수가 잘 만들어졌는지 확인하기
item_code = "373220"
item_name = "LG에너지솔루션"

get_days_list(item_code)

1 2 3 4 5 6 7 8 

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.05.17,408500.0,8000.0,400500.0,411000.0,399000.0,228839.0
1,2022.05.16,400500.0,0.0,407000.0,407500.0,397000.0,200731.0
2,2022.05.13,400500.0,12500.0,390500.0,401500.0,390500.0,228636.0
3,2022.05.12,388000.0,3500.0,390000.0,395500.0,386000.0,251570.0
4,2022.05.11,391500.0,2000.0,394000.0,398000.0,391500.0,239355.0
...,...,...,...,...,...,...,...
68,2022.02.07,548000.0,44000.0,520000.0,548000.0,511000.0,1911176.0
69,2022.02.04,504000.0,27000.0,476500.0,505000.0,476000.0,2088996.0
70,2022.02.03,477000.0,27000.0,458000.0,495500.0,441000.0,2918435.0
71,2022.01.28,450000.0,55000.0,476000.0,483000.0,445000.0,4559773.0


## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [171]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = get_days_list(item_code)
df

1 2 3 4 5 6 7 8 

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.05.17,408500.0,8000.0,400500.0,411000.0,399000.0,228952.0
1,2022.05.16,400500.0,0.0,407000.0,407500.0,397000.0,200731.0
2,2022.05.13,400500.0,12500.0,390500.0,401500.0,390500.0,228636.0
3,2022.05.12,388000.0,3500.0,390000.0,395500.0,386000.0,251570.0
4,2022.05.11,391500.0,2000.0,394000.0,398000.0,391500.0,239355.0
...,...,...,...,...,...,...,...
68,2022.02.07,548000.0,44000.0,520000.0,548000.0,511000.0,1911176.0
69,2022.02.04,504000.0,27000.0,476500.0,505000.0,476000.0,2088996.0
70,2022.02.03,477000.0,27000.0,458000.0,495500.0,441000.0,2918435.0
71,2022.01.28,450000.0,55000.0,476000.0,483000.0,445000.0,4559773.0


In [147]:
%timeit df.iloc[0,0]

37.5 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [148]:
%timeit df.iloc[0][0]

144 µs ± 7.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [149]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.05.17,408500.0,8000.0,400500.0,411000.0,399000.0,228952.0
1,2022.05.16,400500.0,0.0,407000.0,407500.0,397000.0,200731.0
2,2022.05.13,400500.0,12500.0,390500.0,401500.0,390500.0,228636.0
3,2022.05.12,388000.0,3500.0,390000.0,395500.0,386000.0,251570.0
4,2022.05.11,391500.0,2000.0,394000.0,398000.0,391500.0,239355.0


In [150]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
68,2022.02.07,548000.0,44000.0,520000.0,548000.0,511000.0,1911176.0
69,2022.02.04,504000.0,27000.0,476500.0,505000.0,476000.0,2088996.0
70,2022.02.03,477000.0,27000.0,458000.0,495500.0,441000.0,2918435.0
71,2022.01.28,450000.0,55000.0,476000.0,483000.0,445000.0,4559773.0
72,2022.01.27,505000.0,92000.0,597000.0,598000.0,450000.0,15946992.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [ ]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
# df["종목코드"]
# df["종목명"]

In [172]:
df['종목코드'] = item_code
df['종목명'] = item_name
df.head(1)

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
0,2022.05.17,408500.0,8000.0,400500.0,411000.0,399000.0,228952.0,373220,LG에너지솔루션


## 컬럼 순서 변경하기

In [ ]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.

In [173]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]
print(df.shape)
df.head(1)

(73, 9)


,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,373220,LG에너지솔루션,2022.05.17,408500.0,8000.0,400500.0,411000.0,399000.0,228952.0


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [174]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
print(df.shape)
df = df.drop_duplicates('날짜')
print(df.shape)

(73, 9)
(73, 9)


## 기술통계값 구하기

In [161]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,73.000000,73.000000,73.000000,73.000000,73.000000,7.300000e+01
mean,430034.246575,11102.739726,431684.931507,439404.109589,421534.246575,9.898333e+05
std,36285.933222,14639.169040,41067.653097,42785.775928,32119.820028,1.957457e+06
min,359500.000000,0.000000,363000.000000,367500.000000,355000.000000,1.156520e+05
25%,408000.000000,2500.000000,406000.000000,412000.000000,399000.000000,3.207460e+05
50%,433000.000000,6000.000000,433500.000000,437500.000000,423500.000000,4.155240e+05
75%,442000.000000,13000.000000,443500.000000,448500.000000,437500.000000,8.954600e+05
max,548000.000000,92000.000000,597000.000000,598000.000000,524000.000000,1.594699e+07


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [ ]:
# 거래량의 과학적 기수법 읽기


## 최근 날짜 구해서 파일명 만들기

In [176]:
# 날짜 column의 첫 row값 확인
# date
df[df['날짜'] == df['날짜'][0]]

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,373220,LG에너지솔루션,2022.05.17,408500.0,8000.0,400500.0,411000.0,399000.0,228952.0


In [ ]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
# file_name

## 파일로 저장하기

In [ ]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.


In [ ]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.


## 전체 과정을 하나의 함수로 만들기

In [181]:
def get_item_list(item_code, item_name):
    df = get_days_list(item_code)
    date = df['날짜'][0]
    filename = f'{item_name}-{item_code}-{date}'
    print(filename)
    df.to_csv(filename)
    return df

In [182]:
item_code = "323410"
item_name = "카카오뱅크"
get_item_list(item_code, item_name)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 카카오뱅크-323410-2022.05.17


,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.05.17,40000.0,1150.0,38450.0,40250.0,38450.0,1148434.0
1,2022.05.16,38850.0,400.0,38500.0,39750.0,38450.0,1189121.0
2,2022.05.13,38450.0,500.0,37850.0,38550.0,37100.0,1708351.0
3,2022.05.12,37950.0,2150.0,39500.0,39600.0,37550.0,2795518.0
4,2022.05.11,40100.0,50.0,39900.0,40300.0,39600.0,957524.0
...,...,...,...,...,...,...,...
185,2021.08.12,73800.0,600.0,73600.0,75400.0,72700.0,5878196.0
186,2021.08.11,74400.0,3000.0,71200.0,77500.0,68100.0,21592734.0
187,2021.08.10,71400.0,7100.0,81800.0,85600.0,68000.0,26495122.0
188,2021.08.09,78500.0,8700.0,69800.0,89100.0,69000.0,44833696.0


In [183]:
pd.read_csv('카카오뱅크-323410-2022.05.17')

,Unnamed: 0,날짜,종가,전일비,시가,고가,저가,거래량
0,0,2022.05.17,40000.0,1150.0,38450.0,40250.0,38450.0,1148434.0
1,1,2022.05.16,38850.0,400.0,38500.0,39750.0,38450.0,1189121.0
2,2,2022.05.13,38450.0,500.0,37850.0,38550.0,37100.0,1708351.0
3,3,2022.05.12,37950.0,2150.0,39500.0,39600.0,37550.0,2795518.0
4,4,2022.05.11,40100.0,50.0,39900.0,40300.0,39600.0,957524.0
...,...,...,...,...,...,...,...,...
185,185,2021.08.12,73800.0,600.0,73600.0,75400.0,72700.0,5878196.0
186,186,2021.08.11,74400.0,3000.0,71200.0,77500.0,68100.0,21592734.0
187,187,2021.08.10,71400.0,7100.0,81800.0,85600.0,68000.0,26495122.0
188,188,2021.08.09,78500.0,8700.0,69800.0,89100.0,69000.0,44833696.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [ ]:
# 도움말 보기
# get_item_list?


In [ ]:
# 소스코드 보기
# get_item_list??
